In [8]:
import pandas as pd 
import numpy as np 
import anndata as an 
import sys, os 
sys.path.append('../src')

import sys
import os
import pathlib 
from typing import *

import torch
import numpy as np 
import pandas as pd 
import anndata as an
from functools import cached_property

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.preprocessing import LabelEncoder 

from models.lib.data import *
from models.lib.lightning_train import *
from data.downloaders.external_download import *

In [21]:
class TESTMODULE(pl.LightningDataModule):
    def __init__(
        self, 
        class_label: str,
        datafiles: List[str]=None,
        labelfiles: List[str]=None,
        urls: Dict[str, List[str]]=None,
        unzip: bool=False,
        sep: str='\t',
        datapath: str=None,
        is_assumed_numeric: bool=True,
        *args,
        **kwargs,
    ):
        super().__init__()

        if urls is not None and datafiles is not None or urls is not None and labelfiles is not None:
            raise ValueError("Either a dictionary of data to download, or paths to datafiles and labelfiles are supported, but not both.")

        self.class_label = class_label
        self.datafiles = datafiles 
        self.labelfiles = labelfiles 

        self.urls = urls 
        self.unzip = unzip 
        self.sep = sep 
        self.datapath = (
            datapath if datapath is not None else join(here, '..', '..', '..', 'data', 'raw')
        )
        
        self.is_assumed_numeric = is_assumed_numeric

        self.args = args 
        self.kwargs = kwargs
        
    def prepare_data(self):
        if self.urls is not None:
            download_raw_expression_matrices(
                self.urls,
                unzip=self.unzip,
                sep=self.sep,
                datapath=self.datapath,
            )
        
        if not self.is_assumed_numeric:
            if self.urls is not None:
                # Since we specifically formulated this download format in download_raw_expression_matrices
                labelfiles = [join(self.datapath, f'labels_{file}') for file in self.urls.keys()]
            else:
                labelfiles = self.labelfiles 
            
            unique_targets = list(
                set(np.concatenate([pd.read_csv(df, sep=self.sep).loc[:, self.class_label].unique() for df in labelfiles]))
            )
            
            le = LabelEncoder()
            le = le.fit(unique_targets)
            
            for file in labelfiles:
                labels = pd.read_csv(file, sep=self.sep)

                labels = labels.rename(
                    columns={
                        self.class_label: f'categorical_{self.class_label}'
                    }
                )
                
                display(labels)
                
                labels.loc[:, self.class_label] = le.transform(
                    labels.loc[:,f'categorical_{self.class_label}']
                )

                labels.to_csv(file, index=False) # Don't need to re-index here 

In [22]:
urls = {
    'human_dental.tsv': [
        'https://cells.ucsc.edu/dental-cells/human-adult-molars/exprMatrix.tsv.gz',
        'https://cells.ucsc.edu/dental-cells/human-adult-molars/meta.tsv'
    ]
}

module = TESTMODULE(
    urls=urls,
    class_label='cell_type',
    is_assumed_numeric=False,
)

In [23]:
module.prepare_data()

https://cells.ucsc.edu/dental-cells/human-adult-molars/exprMatrix.tsv.gz https://cells.ucsc.edu/dental-cells/human-adult-molars/meta.tsv


,cell_id,cluster,categorical_cell_type,cluster_celltype,sample_source
0,human_germinectomy_1_one_AAACCTGTCAACTCTT-1,5,pulp,5 - pulp,human_germinectomy
1,human_germinectomy_1_one_AAACGGGCATTGCGGC-1,5,pulp,5 - pulp,human_germinectomy
2,human_germinectomy_1_one_AAACGGGGTGCCTGTG-1,13,pulp,13 - pulp,human_germinectomy
3,human_germinectomy_1_one_AAAGATGTCACTCTTA-1,9,pulp,9 - pulp,human_germinectomy
4,human_germinectomy_1_one_AAAGCAACAACCGCCA-1,15,other immune cells,15 - other immune cells,human_germinectomy
...,...,...,...,...,...
41668,No5_24_yo_healthy_retained_one_TTTGTTGGTCGTCATA-1,1,pulp,1 - pulp,No5_24_yo_healthy_retained
41669,No5_24_yo_healthy_retained_one_TTTGTTGGTCTGTGCG-1,7,glial,7 - glial,No5_24_yo_healthy_retained
41670,No5_24_yo_healthy_retained_one_TTTGTTGGTTATGTCG-1,9,pulp,9 - pulp,No5_24_yo_healthy_retained
41671,No5_24_yo_healthy_retained_one_TTTGTTGTCGCAGTTA-1,9,pulp,9 - pulp,No5_24_yo_healthy_retained


SyntaxError: cannot assign to operator (4255437041.py, line 1)

In [14]:
import pathlib 

pathlib.Path('test.csv').stem

'test'

In [20]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [21]:
t = pd.Series([4,5,6], [1,2,3])
t

1    4
2    5
3    6
dtype: int64

In [22]:
le.fit_transform(t)

array([0, 1, 2])